In [32]:
import pandas as pd
from ucimlrepo import fetch_ucirepo 

### Extracting the data: 

We will import four datasets and combine them into one DataFrame. We will fetch the Cleveland data using the `ucimlrepo` library first, because this dataset contains the proper column headers, and the other three datasets do not. Next, we will import the data from the other three locations-- Hungary, Switzerland, and VA Long Beach-- which we downloaded as .data files from the UC repo. Finally, we will combine the four locations into one dataset.

In [33]:
# Get the Cleveland dataset from the UC repo

# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 
  
# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets  

In [34]:
# Create a dataframe to house all Cleveland variables
df_cleveland = pd.concat([X,y], axis=1)
df_cleveland.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


In [35]:
# Create dataframes for other three locations
df_hungarian = pd.read_csv('Resources/processed.hungarian.data', header=None)
df_switzerland = pd.read_csv('Resources/processed.switzerland.data', header=None)
df_va = pd.read_csv('Resources/processed.va.data', header=None)

In [36]:
# Combine these three locations' dataframes into one dataframe
data3 = pd.concat([df_hungarian, df_switzerland, df_va], ignore_index=True)
print(data3.shape)
data3.head()

(617, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,28,1,2,130,132,0,2,185,0,0.0,?,?,?,0
1,29,1,2,120,243,0,0,160,0,0.0,?,?,?,0
2,29,1,2,140,?,0,0,170,0,0.0,?,?,?,0
3,30,0,1,170,237,0,1,170,0,0.0,?,?,6,0
4,31,0,2,100,219,0,1,150,0,0.0,?,?,?,0


In [37]:
# Rename the columns for 3 additional locations in preparation to use .concat()
data3.columns = df_cleveland.columns
data3.columns

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'num'],
      dtype='object')

In [38]:
# Create our combined dataset of all four locations

heart_data0 = pd.concat([df_cleveland, data3])
print(heart_data0.shape)
heart_data0.head()

(920, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0,0


#### Transform:

The dataset contains a categorical variable, `num`, which indicates the presence and severity of heart disease on a scale of 0 to 4. For the purpose of our analysis, we will convert this to a binary variable that indicates the presence of heart disease.

In [39]:
# Check the target variable before conversion
print(heart_data0['num'].isna().sum())
print(heart_data0['num'].value_counts())

0
num
0    411
1    265
2    109
3    107
4     28
Name: count, dtype: int64


In [40]:
# Convert target variable column from heart disease severity to binary presence
for index, row in heart_data0.iterrows():
    if row['num'] > 0:
        heart_data0.loc[index, 'num'] = 1
    else:
        heart_data0.loc[index, 'num'] = 0

In [41]:
# Confirm the column conversion
heart_data0['num'].value_counts()

num
1    484
0    436
Name: count, dtype: int64

Next, we will handle missing values. 

Many cells contain a "?" as an entry, which is not detected as a missing value, so we will convert cells with ?'s to na's.

In [42]:
# Create a backup copy of dataset prior to further transformations
heart_data = heart_data0.copy()
heart_data.shape

(920, 14)

In [43]:
# Checking ?s:
# Most ?s are located in slope, ca, and thal columns
heart_data0.tail()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
612,54,0,4,127,333,1,1,154,0,0,?,?,?,1
613,62,1,1,?,139,0,1,?,?,?,?,?,?,0
614,55,1,4,122,223,1,1,100,0,0,?,?,6,1
615,58,1,4,?,385,1,2,?,?,?,?,?,?,0
616,62,1,2,120,254,0,2,93,1,0,?,?,?,1


In [44]:
# Determine missing values
heart_data0.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          4
thal        2
num         0
dtype: int64

In [45]:
# Replace the ?'s to clean the dataset
heart_data0 = heart_data0.replace({'?': None})
heart_data0 = heart_data0.dropna()
heart_data0.shape

(299, 14)

If we designate ?s as na's and then use .dropna(), our number of observations falls to just 299, which is hardly better than just using the Cleveland dataset in isolation. Instead, we will drop the two most problematic columns, `ca` and `thal`, before using the .replace() function, to keep more observations instact.

In [46]:
# Use original dataset copy
print(heart_data.shape)

# Check for problematic data points
heart_data['slope'].value_counts()

(920, 14)


slope
?    309
2    205
1    142
2    140
1     61
3     42
3     21
Name: count, dtype: int64

In [47]:
# Check for problematic data points
heart_data['ca'].value_counts()

ca
?      607
0.0    176
1.0     65
2.0     38
3.0     20
0        5
2        3
1        2
Name: count, dtype: int64

In [48]:
# Check for problematic data points
heart_data['thal'].value_counts()

thal
?      484
3.0    166
7.0    117
7       75
3       30
6       28
6.0     18
Name: count, dtype: int64

In [49]:
# Drop the two problematic columns
heart_data.drop(columns=['ca','thal'], inplace=True)
heart_data.shape

(920, 12)

In [50]:
# Replace the ?'s to clean the dataset
heart_data = heart_data.replace({'?': None})
heart_data = heart_data.dropna()
heart_data.shape

(531, 12)

Continue cleaning the data: 

Some columns contain cells with identical values that are being read as two separate numbers, as we saw above.

In [51]:
# Create a backup copy prior to further tranformation
heart_data1 = heart_data.copy()

In [52]:
# Check column for value issues
heart_data['slope'].value_counts()

slope
2    170
1    142
2    140
1     31
3     27
3     21
Name: count, dtype: int64

In [53]:
# Investigate the discrepancy

# Some entries are strings
heart_data['slope'].unique()

array([3, 2, 1, '2', '1', '3'], dtype=object)

In [54]:
# Convert all cells in column to integer
heart_data['slope'] = heart_data['slope'].astype(int)
heart_data['slope'].unique()

array([3, 2, 1])

In [55]:
# Check data types of other columns for possible issues
heart_data.dtypes

age          int64
sex          int64
cp           int64
trestbps    object
chol        object
fbs         object
restecg     object
thalach     object
exang       object
oldpeak     object
slope        int64
num          int64
dtype: object

In [56]:
# Check a column with dtype object
# This column has the same issue with strings
heart_data['trestbps'].unique()

array([145, 160, 120, 130, 140, 172, 150, 110, 132, 117, 135, 112, 105,
       124, 125, 142, 128, 170, 155, 104, 180, 138, 108, 134, 122, 115,
       118, 100, 200, 94, 165, 102, 152, 101, 126, 174, 148, 178, 158,
       192, 129, 144, 123, 136, 146, 106, 156, 154, 114, 164, '112',
       '120', '110', '140', '108', '150', '130', '124', '160', '180',
       '100', '92', '125', '122', '200', '138', '135', '145', '170',
       '136', '155', '115', '105', '185', '132', '142', '128', '126',
       '116', '0', '144', '152', '146', '158', '96', '134', '156', '118',
       '190', '114'], dtype=object)

In [57]:
# Convert datatypes of object columns 
heart_data[['trestbps','chol','fbs','restecg','thalach','exang']] = heart_data[['trestbps','chol','fbs','restecg','thalach','exang']].astype(int)
heart_data['oldpeak'] = heart_data['oldpeak'].astype(float)
heart_data.dtypes

age           int64
sex           int64
cp            int64
trestbps      int64
chol          int64
fbs           int64
restecg       int64
thalach       int64
exang         int64
oldpeak     float64
slope         int64
num           int64
dtype: object

Continue cleaning the data: check for outliers and additional possible issues

In [58]:
# Check the values of each column for successful conversion and additional possible problems
for column in heart_data.columns:
    unique_values = heart_data[column].unique()
    print(f"Column '{column}': {unique_values}")

Column 'age': [63 67 37 41 56 62 57 53 44 52 48 54 49 64 58 60 50 66 43 40 69 59 42 55
 61 65 71 51 46 45 39 68 47 34 35 29 70 77 38 74 76 36 31 33 73 72 75]
Column 'sex': [1 0]
Column 'cp': [1 4 3 2]
Column 'trestbps': [145 160 120 130 140 172 150 110 132 117 135 112 105 124 125 142 128 170
 155 104 180 138 108 134 122 115 118 100 200  94 165 102 152 101 126 174
 148 178 158 192 129 144 123 136 146 106 156 154 114 164  92 185 116   0
  96 190]
Column 'chol': [233 286 229 250 204 236 268 354 254 203 192 294 256 263 199 168 239 275
 266 211 283 284 224 206 219 340 226 247 167 230 335 234 177 276 353 243
 225 302 212 330 175 417 197 198 290 253 172 273 213 305 216 304 188 282
 185 232 326 231 269 267 248 360 258 308 245 270 208 264 321 274 325 235
 257 164 141 252 255 201 222 260 182 303 265 309 307 249 186 341 183 407
 217 288 220 209 227 261 174 281 221 205 240 289 318 298 564 246 322 299
 300 293 277 214 207 223 160 394 184 315 409 244 195 196 126 313 259 200
 262 215 228 193 271 210 

In [59]:
# Resting bloodpressure and cholesterol should not be 0
# Remove these and confirm the change

heart_data = heart_data.drop(heart_data[heart_data['trestbps'] == 0].index)
heart_data = heart_data.drop(heart_data[heart_data['chol'] == 0].index)
heart_data.reset_index(drop=True, inplace=True)
heart_data[heart_data['chol'] == 0]

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,num


In [60]:
# See how dropping 0's impacted our number of observations
print(heart_data.shape)
heart_data.head()

(460, 12)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0
1,67,1,4,160,286,0,2,108,1,1.5,2,0
2,67,1,4,120,229,0,2,129,1,2.6,2,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0


In [61]:
# Check for duplicated rows
heart_data.duplicated().sum()

0

In [62]:
# Export transformed DataFrame to csv for SQL
heart_data.to_csv("Resources/heart_data.csv", index=False)